# Packages

In [63]:
import pandas as pd
import numpy as np
import math
import sqlalchemy 
from sqlalchemy import create_engine, text
from routingpy import ORS as ors
import requests

from db_secrets import SQL_107
from ors_secrets import get_ors_key

# Connection

In [64]:
## text for gp query
gp_query_text = """
SELECT p.[Organisation_Code] AS	[GP Practice Code]
      ,p.[Organisation_Name] AS [GP Practice Name]
	  ,p.[Postcode]
FROM [UK_Health_Dimensions].[ODS].[GP_Practices_And_Prescribing_CCs_SCD] AS p
WHERE 
	1=1
	AND	p.[Is_Latest] = 1
	AND	p.[High_Level_Health_Authority_Code]  = 'QHM'
	AND	p.[Prescribing_Setting] = 4
	AND	p.[Parent_Organisation_Code] != '01H'
"""

In [65]:
## text for site query
ae_query_text = """
-- removes messages
SET NOCOUNT ON

-- Creates lookup for UCC sites
DROP TABLE IF EXISTS #RXP_UCC
CREATE TABLE #RXP_UCC (
		[ods_code] varchar(5)
	,	[site_name] varchar(250)
	);
INSERT INTO #RXP_UCC ([ods_code],[site_name])
VALUES
	 ('RXPRD','Seaham Urgent Care Centre'			   )
	,('RXP09','Peterlee Urgent Care Centre'			   )
	,('RXP09','Peterlee UTC'						   )
	,('RXPCP','OOH University Hospital of North Durham')
	,('RXPCP','Durham UTC'							   )
	,('RXPDA','Darlington UTC'						   )
	,('RXPDA','Darlington Out of Hours Service'		   )
	,('RXPBA','Bishop Auckland Urgent Care Centre'	   )
	,('RXPBA','Bishop Auckland UTC'					   )
	,('RXP11','Shotley Bridge UTC'					   )
	,('RXP11','OOH Shotley Bridge Urgent Care Centre'  );

-- Gets postcodes
SELECT DISTINCT 
		a.[ods_code]			AS	[UEC Site Code]
	,	t.[Organisation_Name]	AS	[UEC Site Name]
	,	t.[Postcode]
FROM 
	(	SELECT DISTINCT
			[attendance.location.site] AS [ods_code]
		FROM
			[LocalDataNECS].[ecds].[emergency_care]	AS a
		WHERE
			1=1
			AND a.[attendance.location.department_type] in ('01','02','03','04')
			AND a.[attendance.location.hes_provider_3] in ('RTD','RR7','RTF','RXP','RVW','RTR','R0B','RNN')
			AND a.[attendance.arrival.date] >= '2022-01-1'
		UNION ALL
		SELECT
			[ods_code]
		FROM 
			#RXP_UCC AS u) AS a
	LEFT JOIN	[UK_Health_Dimensions].[ODS].[NHS_Trusts_And_Trust_Sites_SCD] AS t
			ON	t.[Is_Latest] = 1
			AND a.[ods_code] = t.[Organisation_Code];

DROP TABLE IF EXISTS #RXP_UCC;
"""

In [66]:
## Create an engine + connection
engine = create_engine(SQL_107())
conn = engine.connect()

## Return data
df_gp_raw = pd.read_sql(gp_query_text,conn)
df_ae_raw = pd.read_sql(ae_query_text,conn)


In [67]:
df_gp = df_gp_raw.copy()
df_ae = df_ae_raw.copy()

# Lat / Long

## API to get lat/long

In [68]:
def fetch_lat_lon(data, col):
    
    # Convert the postcodes from the DataFrame into a list
    postcodes = data[col].tolist()

    # to split data into 100s
    n = round(data[col].count()/100)

    # results list
    results = []

    for x in range(0,n+1):
        
        # 100 rows
        start = x*100
        end = (x*100)+100
        
        # Prepare the payload for the POST request
        payload = {"postcodes": postcodes[start:end]}
        
        # Make the POST request to the API
        response = requests.post("https://api.postcodes.io/postcodes", json=payload)
        
        # Check if the response is successful
        if response.status_code == 200:
            print(f'Status 200. Fetched rows: {start} to {end-1}')

            # Parse the JSON response
            response_data = response.json()
            
            # Extract latitude and longitude from each result

            for i in response_data['result']:
                if i['result']:  # Ensure there is a valid result
                    lat = i['result']['latitude']
                    lon = i['result']['longitude']
                    results.append({"Postcode": i['query']
                                    , "latitude": lat, "longitude": lon})
                else:
                    results.append({"Postcode": i['query']
                                    , "latitude": None, "longitude": None})
        else:
            break

    # checks if last response was sucessful
    if response.status_code == 200:           
        # Convert results into a DataFrame
        results_df = pd.DataFrame(results).drop_duplicates()

        # joins results back into data
        data = data.merge(results_df, how = 'left'
                            ,left_on=col, right_on='Postcode')
        return(data)
    
    # print error message if api fails
    else:
        print("Failed to fetch data:", response.status_code)
        return None

## Fetch lat/long

In [69]:
# Fetch lat/lon for each postcode in the DataFrame
df_gp_geo = fetch_lat_lon(df_gp,'Postcode')
df_ae_geo = fetch_lat_lon(df_ae,'Postcode')

Status 200. Fetched rows: 0 to 99
Status 200. Fetched rows: 100 to 199
Status 200. Fetched rows: 200 to 299
Status 200. Fetched rows: 300 to 399
Status 200. Fetched rows: 0 to 99


In [70]:
df_gp_geo.head()

,GP Practice Code,GP Practice Name,Postcode,latitude,longitude
0,A83626,EVENWOOD MEDICAL PRACTICE,DL14 9SU,54.620132,-1.758629
1,A81022,HILLSIDE PRACTICE,TS12 2TG,54.563154,-0.980814
2,A81058,COULBY MEDICAL PRACTICE,TS8 0TL,54.524805,-1.213762
3,A83057,EAST DURHAM MEDICAL GROUP,TS28 5PZ,54.725581,-1.378951
4,A83029,GREAT LUMLEY SURGERY,DH3 4LE,54.837249,-1.543118


In [71]:
df_ae_geo.head()

,UEC Site Code,UEC Site Name,Postcode,latitude,longitude
0,R0B01,SUNDERLAND ROYAL HOSPITAL,SR4 7TP,54.902214,-1.410327
1,RXPCP,UNIVERSITY HOSPITAL OF NORTH DURHAM,DH1 5TW,54.788497,-1.593834
2,RXPDA,DARLINGTON MEMORIAL HOSPITAL,DL3 6HX,54.530374,-1.563733
3,RNN62,CUMBERLAND INFIRMARY,CA2 7HY,54.896504,-2.957796
4,RXP09,PETERLEE COMMUNITY HOSPITAL,SR8 5UQ,54.754067,-1.333039


# Routing

In [29]:
ors_api = ors(api_key=get_ors_key())

In [58]:
## Makes list for sources (gp) and destinations (ae)
list_gp_geo = df_gp_geo[['longitude','latitude']].values.tolist()
list_ae_geo = df_ae_geo[['longitude','latitude']].values.tolist()

## Length of lists
len_gp = len(list_gp_geo)
len_ae = len(list_ae_geo)

## Max 3500 per API call, so split the data into chunks of that size
iter = math.ceil((len_gp * len_ae)/3500)

## List to hold results
matrix_results = []

## Iterate over the sites
for x in range(0,iter):
    
    ## Size of an iteration of destinations
    y = math.ceil(len_ae/iter)
    
    ## print for response
    print(f'{x} UEC sites: {x*y} to {(x*y)+y}')
    
    ## short list of lat/long for destinations (ae)
    shortlist_ae_geo = [list_ae_geo[i] for i in range(x*y,(x*y)+y)]
    short_len_ae = len(shortlist_ae_geo)
    
    ## Overall list
    list_geo = list_gp_geo + shortlist_ae_geo
    
    ## Positions of sources and destinations to use
    sources_list=list(range(0,len_gp))
    destinations_list=list(range(len_gp,len_gp+short_len_ae))

    ## API call
    distance_matrix = ors_api.matrix(
                locations=list_geo
                ,profile='driving-car'
                ,sources=sources_list
                ,destinations=destinations_list            
                ,dry_run = False
            )
    
    ## Takes the results and puts it into a dataframe
    for i, source in enumerate(list_gp_geo):
        for j, destination in enumerate(shortlist_ae_geo):
            matrix_results.append({
                "gp_long": source[0],
                "gp_lat": source[1],
                "ae_long": destination[0],
                "ae_lat": destination[1],
                "duration": distance_matrix.durations[i][j]
            })

matrix_df = pd.DataFrame(matrix_results)

0 UEC sites: 0 to 8
1 UEC sites: 8 to 16
2 UEC sites: 16 to 24
3 UEC sites: 24 to 32


## Wrangle results

In [72]:
df_times = pd.merge(
    matrix_df
    ,df_gp_geo
    ,left_on=["gp_lat", "gp_long"]
    ,right_on=["latitude", "longitude"]
    ,how="left"
    ,suffixes=('','_gp')
    )

df_times = pd.merge(
    df_times
    ,df_ae_geo
    ,left_on=["ae_lat", "ae_long"]
    ,right_on=["latitude", "longitude"]
    ,how="left"
    ,suffixes=('','_ae')
    )

In [76]:
df_times = df_times[[
            'GP Practice Code'
            ,'GP Practice Name'
            ,'UEC Site Code'
            ,'UEC Site Name'
            ,'duration']].copy()

# Export data

In [79]:
df_times.to_csv('gp_to_uec_travel_durations.csv', index=False)